In [1]:
#### IMPORTS

import re
import abc
import operator
import numpy as np
from collections import Counter, defaultdict, namedtuple, deque, abc
from itertools   import (permutations, combinations, chain, cycle, product, islice, 
                         takewhile, zip_longest, starmap, count as count_from)
from functools   import lru_cache, reduce
from heapq import (heappush, heappop, nlargest, nsmallest)

from pprint import pprint as p, pformat as pf
import toolz.curried as t
from tqdm import tqdm_notebook as tq
from dataclasses import dataclass

#### CONSTANTS

alphabet = 'abcdefghijklmnopqrstuvwxyz'
ALPHABET = alphabet.upper()
infinity = float('inf')

#### SIMPLE UTILITY FUNCTIONS

cat = ''.join

def ints(start, end, step=1):
    "The integers from start to end, inclusive: range(start, end+1)"
    return range(start, end + 1, step)

def first(iterable, default=None): 
    "The first item in an iterable, or default if it is empty."
    return next(iter(iterable), default)

def head(iterable, n=5):
    "The first n items in an iterable"
    return tuple(islice(iterable, n))

def tail(iterable, n=1):
    "Skip n items in an iterable"
    return islice(iterable, n, None)

def first_true(iterable, pred=None, default=None):
    """Returns the first true value in the iterable.
    If no true value is found, returns *default*
    If *pred* is not None, returns the first item
    for which pred(item) is true."""
    # first_true([a,b,c], default=x) --> a or b or c or x
    # first_true([a,b], fn, x) --> a if fn(a) else b if fn(b) else x
    return next(filter(pred, iterable), default)

def nth(iterable, n, default=None):
    "Returns the nth item of iterable, or a default value"
    return next(islice(iterable, n, None), default)

def upto(iterable, maxval):
    "From a monotonically increasing iterable, generate all the values <= maxval."
    # Why <= maxval rather than < maxval? In part because that's how Ruby's upto does it.
    return takewhile(lambda x: x <= maxval, iterable)

identity = lambda x: x

def quantify(iterable, pred=bool):
    "Count how many times the predicate is true of an item in iterable."
    return sum(map(pred, iterable))

def multimap(items):
    "Given (key, val) pairs, return {key: [val, ....], ...}."
    result = defaultdict(list)
    for (key, val) in items:
        result[key].append(val)
    return result

def overlapping(iterable, n):
    """Generate all (overlapping) n-element subsequences of iterable.
    overlapping('ABCDEFG', 3) --> ABC BCD CDE DEF EFG"""
    if isinstance(iterable, abc.Sequence):
        yield from (iterable[i:i+n] for i in range(len(iterable) + 1 - n))
    else:
        result = deque(maxlen=n)
        for x in iterable:
            result.append(x)
            if len(result) == n:
                yield tuple(result)
                
def pairwise(iterable):
    "s -> (s0,s1), (s1,s2), (s2, s3), ..."
    return overlapping(iterable, 2)

def mapt(fn, *args): 
    "Do a map, and make the results into a tuple."
    return tuple(map(fn, *args))

def map2d(fn, grid):
    "Apply fn to every element in a 2-dimensional grid."
    return tuple(mapt(fn, row) for row in grid)

def flatmap(fn, *args):
    "Do a map and a one-level flatten"
    return tuple(chain.from_iterable(map(fn, *args)))

def repeat(n, fn, arg, *args, **kwds):
    "Repeat arg = fn(arg) n times, return arg."
    return nth(repeatedly(fn, arg, *args, **kwds), n)

def repeatedly(fn, arg, *args, **kwds):
    "Yield arg, fn(arg), fn(fn(arg)), ..."
    yield arg
    while True:
        arg = fn(arg, *args, **kwds)
        yield arg
        
def repeatedly1(fn, arg, *args, **kwds):
    "Yield fn(arg), fn(fn(arg)), ..."
    return tail(repeatedly(fn, arg, *args, **kwds))

def compose(f, g): 
    "The function that computes f(g(x))."
    return lambda x: f(g(x))

#### FILE INPUT AND PARSING

def Input(day, line_parser=str.strip, test=False, file_template='data/2019/{}.txt'):
    "For this day's input file, return a tuple of each line parsed by `line_parser`."
    return mapt(line_parser, open(file_template.format(
        f'{day}test' if test else day
    )))

@t.curry
def Tokens(line, sep=','):
    "Splits line into delimited tokens"
    return line.strip().split(sep)

def integers(text): 
    "A tuple of all integers in a string (ignore other characters)."
    return mapt(int, re.findall(r'-?\d+', text))

def digits(number):
    "Tuple of digits in number"
    return mapt(int, str(number))

################ 2-D points implemented using (x, y) tuples

def X(point): return point[0]
def Y(point): return point[1]
def Z(point): return point[2]

origin = (0, 0)
HEADINGS = UP, LEFT, DOWN, RIGHT = (0, -1), (-1, 0), (0, 1), (1, 0)

def turn_right(heading): return HEADINGS[HEADINGS.index(heading) - 1]
def turn_around(heading):return HEADINGS[HEADINGS.index(heading) - 2]
def turn_left(heading):  return HEADINGS[HEADINGS.index(heading) - 3]

def add(A, B): 
    "Element-wise addition of two n-dimensional vectors."
    return mapt(sum, zip(A, B))

def sub(A, B): 
    "Element-wise subtraction of two n-dimensional vectors."
    return tuple(a - b for a, b in zip(A, B))

def neighbors4(point): 
    "The four neighboring squares."
    x, y = point
    return (          (x, y-1),
            (x-1, y),           (x+1, y), 
                      (x, y+1))

def neighbors8(point): 
    "The eight neighboring squares."
    x, y = point 
    return ((x-1, y-1), (x, y-1), (x+1, y-1),
            (x-1, y),             (x+1, y),
            (x-1, y+1), (x, y+1), (x+1, y+1))

def cityblock_distance(P, Q=origin): 
    "Manhatten distance between two points."
    return sum(abs(p - q) for p, q in zip(P, Q))

def distance(P, Q=origin): 
    "Straight-line (hypotenuse) distance between two points."
    return sum((p - q) ** 2 for p, q in zip(P, Q)) ** 0.5

def king_distance(P, Q=origin):
    "Number of chess King moves between two points."
    return max(abs(p - q) for p, q in zip(P, Q))

################ Debugging 

def trace1(f):
    "Print a trace of the input and output of a function on one line."
    def traced_f(*args):
        result = f(*args)
        print('{}({}) = {}'.format(f.__name__, ', '.join(map(str, args)), result))
        return result
    return traced_f

def grep(pattern, iterable):
    "Print lines from iterable that match pattern."
    for line in iterable:
        if re.search(pattern, line):
            print(line)
            
class Struct:
    "A structure that can have any fields defined."
    def __init__(self, **entries): self.__dict__.update(entries)
    def __repr__(self): 
        fields = ['{}={}'.format(f, self.__dict__[f]) 
                  for f in sorted(self.__dict__)]
        return 'Struct({})'.format(', '.join(fields))

################ A* and Breadth-First Search (tracking states, not actions)

def always(value): return (lambda *args: value)

def Astar(start, moves_func, h_func, cost_func=always(1)):
    "Find a shortest sequence of states from start to a goal state (where h_func(s) == 0)."
    frontier  = [(h_func(start), start)] # A priority queue, ordered by path length, f = g + h
    previous  = {start: None}  # start state has no previous state; other states will
    path_cost = {start: 0}     # The cost of the best path to a state.
    Path      = lambda s: ([] if (s is None) else Path(previous[s]) + [s])
    while frontier:
        (f, s) = heappop(frontier)
        if h_func(s) == 0:
            return Path(s)
        for s2 in moves_func(s):
            g = path_cost[s] + cost_func(s, s2)
            if s2 not in path_cost or g < path_cost[s2]:
                heappush(frontier, (g + h_func(s2), s2))
                path_cost[s2] = g
                previous[s2] = s

def bfs(start, moves_func, goals):
    "Breadth-first search"
    goal_func = (goals if callable(goals) else lambda s: s in goals)
    return Astar(start, moves_func, lambda s: (0 if goal_func(s) else 1))

Completed IntCode interpreter:

In [2]:
ADD, MUL, INPUT, OUTPUT, JT, JF, LT, EQ, REL, STOP = (
    1, 2, 3, 4, 5, 6, 7, 8, 9, 99)

opcodes = {
    1: 'ADD', 2: 'MUL', 3: 'INPUT', 4: 'OUTPUT',
    5: 'JT', 6: 'JF', 7: 'LT', 8: 'EQ', 9: 'REL',
    99: 'STOP',
}

arities = {
    ADD: 3, MUL: 3, INPUT: 1, OUTPUT: 1,
    JT: 2, JF: 2, LT: 3, EQ: 3, REL: 1,
    STOP: 0
}

def Opcode(val):
    code = val % 100
    return (val % 100,
            [val // 100 % 10, val // 1000 % 10, val // 10000 % 10])

@dataclass
class Comp:
    prog: defaultdict
    inputs: deque
    outputs: deque
    pc: int = 0
    base: int = 0
    waiting: bool = False
    stopped: bool = False
        
    def __init__(self, prog, inputs=[], outputs=[]):
        self.prog = defaultdict(int, dict(enumerate(prog)))
        self.inputs = deque(inputs)
        self.outputs = deque(outputs)
    
    def step(self):
        prog, pc = self.prog, self.pc
        inputs, outputs = self.inputs, self.outputs
        
        code, modes = Opcode(prog[pc])
        arg1 = self.read(pc + 1, modes[0])
        arg2 = self.read(pc + 2, modes[1])
        if code == STOP: self.stopped = True
        if code == ADD: self.write(arg1 + arg2, pc + 3, modes[2])
        elif code == MUL: self.write(arg1 * arg2, pc + 3, modes[2])
        elif code == INPUT:
            if not inputs:
                self.waiting = True
                return
            self.write(inputs.popleft(), pc + 1, modes[0])
        elif code == OUTPUT: outputs.append(arg1)
        elif code == JT and arg1 != 0:
            self.pc = arg2
            return
        elif code == JF and arg1 == 0:
            self.pc = arg2
            return
        elif code == LT: self.write(int(arg1 < arg2), pc + 3, modes[2])
        elif code == EQ: self.write(int(arg1 == arg2), pc + 3, modes[2])
        elif code == REL: self.base += arg1
        self.pc += arities[code] + 1
        
    def read(self, loc, mode):
        '''
        Reads in loc from prog accounting for mode.
        '''
        arg = self.prog[loc]
        return (self.prog[arg]             if mode == 0 else
                arg                        if mode == 1 else
                self.prog[self.base + arg] if mode == 2 else
                None)
    
    def write(self, val, loc, mode):
        '''
        Writes argument to prog accounting for mode.
        '''
        arg = self.prog[loc]
        if mode == 0:
            self.prog[arg] = val
        elif mode == 2:
            self.prog[self.base + arg] = val
        else:
            raise ValueError('Bad mode in write')
            
    def input(self, val):
        self.waiting = False
        self.inputs.append(val)
        return self
    
    def run(self):
        while not self.waiting and not self.stopped:
            self.step()
        return self
    
    def output(self):
        return self.outputs[-1]
    
    def out(self):
        return list(self.outputs)

In [3]:
def print_prog(prog):
    'Simple disassembler for IntCode programs'
    ins = deque(prog)
    pc = 0
    while ins:
        code, modes = Opcode(ins.popleft())
        arity = arities.get(code, 0)
        args = [ins.popleft() for _ in range(arity)]
        print(f'{pc:<3} {opcodes.get(code, code):>8}: {str(args):<20} {modes}')
        pc += arity + 1
# print_prog(prog)

## Day 1

In [172]:
masses = Input(1, line_parser=int)
sum(m // 3 - 2 for m in masses)

3167282

In [175]:
def to_fuel(n):
    return n // 3 - 2

def total_fuel(mass):
    return sum(takewhile(lambda n: n > 0,
                         repeatedly1(to_fuel, mass)))

sum(map(total_fuel, masses))

4748063

## Day 2

In [254]:
orig = list(Input(2, line_parser=integers)[0])
# prog = [1,9,10,3,2,3,11,0,99,30,40,50]
prog = list(orig)
prog[1] = 12
prog[2] = 2
len(prog)

145

In [255]:
def process(pos, prog=prog):
    opcode, reg1, reg2, out = prog[pos:pos+4]
    if opcode == 99:
        return -1
    if opcode == 1:
        prog[out] = prog[reg1] + prog[reg2]
    if opcode == 2:
        prog[out] = prog[reg1] * prog[reg2]
    return pos + 4
first_true(repeatedly(process, 0), pred=lambda x: x is -1)

-1

In [256]:
prog[0]

8017076

In [257]:
def trial(noun, verb, prog=prog):
    prog = list(prog)
    prog[1] = noun
    prog[2] = verb
    pos = 0
    while pos != -1:
        pos = process(pos, prog)
    return prog[0]

goal = 19690720
first((noun, verb)
      for noun in range(100)
      for verb in range(100)
      if trial(noun, verb, orig) == goal)

(31, 46)

## Day 3

In [258]:
def Path(line):
    return [(token[0], int(token[1:])) for token in Tokens(line)]

path1, path2 = Input(3, line_parser=Path)
# path1 = Path('R8,U5,L5,D3')
# path2 = Path('U7,R6,D4,L4')
path1[:5]

[('R', 1000), ('D', 940), ('L', 143), ('D', 182), ('L', 877)]

In [259]:
def along(section, start=origin):
    direction, length = section
    heading = HEADINGS['ULDR'.index(direction)]
    return head(repeatedly1(add, start, heading), n=length)

def step(walk, section):
    start = walk[-1] if walk else origin
    return (*walk, *along(section, start))

step([], ('R', 8))

((1, 0), (2, 0), (3, 0), (4, 0), (5, 0), (6, 0), (7, 0), (8, 0))

In [260]:
walk1 = reduce(step, path1, [])
walk2 = reduce(step, path2, [])
intersections = set(walk1) & set(walk2)
min(mapt(cityblock_distance, intersections))

865

In [261]:
def wire_distance(intersection):
    # Add 2 since we need to account for the origin for two paths
    return walk1.index(intersection) + walk2.index(intersection) + 2

min(mapt(wire_distance, intersections))

35038

## Day 4

In [262]:
my_input = '284639-748759'
my_range = range(*mapt(int, Tokens('284639-748759', sep='-')))
my_range

range(284639, 748759)

In [263]:
def has_repeat(digs):
    return any(i == j for i, j in pairwise(digs))

def is_monotonic(digs):
    return all(i <= j for i, j in pairwise(digs))

def is_password(number):
    digs = digits(number)
    return is_monotonic(digs) and has_repeat(digs)

quantify(my_range, is_password)

895

The monotonic condition ensures that all unique digit values appear together.

In [264]:
def has_single_repeat(digs):
    return any(count == 2 for count in Counter(digs).values())

def is_paironly_password(number):
    digs = digits(number)
    return is_monotonic(digs) and has_single_repeat(digs)

quantify(my_range, is_paironly_password)

591

## Day 5

In [265]:
prog, = Input(5, line_parser=integers)
prog[:5]

(3, 225, 1, 225, 6)

In [266]:
ADD, MUL, INPUT, OUTPUT, JT, JF, LT, EQ, STOP = (
    1, 2, 3, 4, 5, 6, 7, 8, 99)

arities = {
    ADD: 3, MUL: 3, INPUT: 1, OUTPUT: 1,
    JT: 2, JF: 2, LT: 3, EQ: 3,
    STOP: 0
}

def Opcode(val):
    code = val % 100
    return (val % 100,
            [val // 100 % 10, val // 1000 % 10, val // 10000 % 10])
Opcode(1003)

(3, [0, 1, 0])

In [267]:
def read_arg(prog, pc, mode):
    '''
    Reads in argument from prog accounting for mode. Returns None if arg isn't
    readable (e.g. program ends)
    '''
    arg = prog[pc] if 0 <= pc < len(prog) else None
    return (arg       if arg is None or mode == 1 else
            prog[arg] if 0 <= arg < len(prog) else
            None)

read_arg([1002, 4,3,4,33], 0, 0)

In [268]:
def run(prog, inputs):
    prog, inputs = list(prog), iter(inputs)
    outputs = []
    pc = 0
    while True:
        code, modes = Opcode(prog[pc])
        arg1 = read_arg(prog, pc + 1, modes[0])
        arg2 = read_arg(prog, pc + 2, modes[1])
#         print(f'{prog[pc:pc+len(modes) + 1]}')
        if code == STOP:
            return outputs[-1]
        if code == ADD:
            out = prog[pc + 3]
            prog[out] = arg1 + arg2
        elif code == MUL:
            out = prog[pc + 3]
            prog[out] = arg1 * arg2
        elif code == INPUT:
            out = prog[pc + 1]
            prog[out] = next(inputs)
        elif code == OUTPUT:
            val = read_arg(prog, pc + 1, modes[0])
            outputs.append(val)
        elif code == JT:
            if arg1 != 0:
                pc = arg2
                continue
        elif code == JF:
            if arg1 == 0:
                pc = arg2
                continue
        elif code == LT:
            out = prog[pc + 3]
            prog[out] = int(arg1 < arg2)
        elif code == EQ:
            out = prog[pc + 3]
            prog[out] = int(arg1 == arg2)
        else:
            raise ValueError(f'Bad opcode at pc: {pc}')
        pc += arities[code] + 1
run([3,9,7,9,10,9,4,9,99,-1,8], [6])

1

In [269]:
assert run([3,0,4,0,99], [10]) == 10
assert run([3,0,4,0,99], [10]) == 10
assert run([3,9,8,9,10,9,4,9,99,-1,8], [10]) == 0
assert run([3,9,8,9,10,9,4,9,99,-1,8], [8]) == 1
assert run([3,9,7,9,10,9,4,9,99,-1,8], [6]) == 1
assert run([3,9,7,9,10,9,4,9,99,-1,8], [8]) == 0
assert run([3,3,1108,-1,8,3,4,3,99], [8]) == 1
assert run([3,3,1107,-1,8,3,4,3,99], [8]) == 0
assert run(prog, [1]) == 14522484
assert run(prog, [5]) == 4655956

In [270]:
run(prog, [1])

14522484

In [271]:
run(prog, [5])

4655956

## Day 6

In [272]:
edges = Input(6, line_parser=Tokens(sep=')'))
edges[:3]

(['TT5', 'Y6Q'], ['ZBC', 'Z2R'], ['8MQ', '51V'])

In [273]:
def add_edge(tree, edge):
    parent, child = edge
    tree[parent].append(child)
    return tree

def Tree(edges):
    return reduce(add_edge, edges, defaultdict(list))

In [274]:
def sum_depths(tree, node='COM', depth=0):
    children = tree[node]
    if not children:
        return depth
    return (depth
            + sum(sum_depths(tree, child, depth + 1) for child in children))
sum_depths(Tree(edges))

268504

The tree distance from A to B is the sum of their distances to their closest common ancestor.

In [275]:
tree = Tree(edges)
me = bfs('COM', lambda n: tree[n], ['YOU'])
san = bfs('COM', lambda n: tree[n], ['SAN'])

# ^ is a set xor; removes common ancestors
# subtract two since we don't need to transfer from the search nodes
len(set(me) ^ set(san)) - 2

409

## Day 7

In [276]:
prog, = Input(7, line_parser=integers)
prog[:5]

(3, 8, 1001, 8, 10)

In [277]:
def amplify(last_output, phase):
    return run(prog, [phase, last_output])

def thrust(phases):
    return reduce(amplify, phases, 0)

phases = max(permutations(range(5), 5), key=thrust)
thrust(phases)

67023

Part 2 requires partial execution of programs which makes life complicated. I decide to create a Comp class holds program, pc, inputs, and outputs.

In [203]:
@dataclass
class Comp:
    prog: list
    pc: int
    inputs: deque
    outputs: deque
    waiting: bool
    stopped: bool
        
    def __init__(self, prog, pc, inputs, outputs=[]):
        self.prog = list(prog)
        self.pc = pc
        self.inputs = deque(inputs)
        self.outputs = deque(outputs)
        self.waiting = False
        self.stopped = False
    
    def step(self):
        prog, pc = self.prog, self.pc
        inputs, outputs = self.inputs, self.outputs
        
        code, modes = Opcode(prog[pc])
        arg1 = read_arg(prog, pc + 1, modes[0])
        arg2 = read_arg(prog, pc + 2, modes[1])
        if code == STOP:
            self.stopped = True
        if code == ADD:
            out = prog[pc + 3]
            prog[out] = arg1 + arg2
        elif code == MUL:
            out = prog[pc + 3]
            prog[out] = arg1 * arg2
        elif code == INPUT:
            if not inputs:
                self.waiting = True
                return
            out = prog[pc + 1]
            prog[out] = inputs.popleft()
        elif code == OUTPUT:
            val = read_arg(prog, pc + 1, modes[0])
            outputs.append(val)
        elif code == JT and arg1 != 0:
            self.pc = arg2
            return
        elif code == JF and arg1 == 0:
            self.pc = arg2
            return
        elif code == LT:
            out = prog[pc + 3]
            prog[out] = int(arg1 < arg2)
        elif code == EQ:
            out = prog[pc + 3]
            prog[out] = int(arg1 == arg2)
        self.pc += arities[code] + 1
    
    def check_waiting(self):
        if self.inputs:
            self.waiting = False

In [379]:
def System(phases):
    system = deque([Comp(prog, 0, [phase]) for phase in phases])
    system[0].inputs.append(0)
    return system

def finished(system):
    return all(comp.stopped for comp in system)

def output(comp):
    return comp.outputs[-1]

def thrust(system):
    return output(system[0])

def run_one_output(system):
    comp = system[0]
    comp.check_waiting()
    while not (comp.stopped or comp.waiting):
        comp.step()
    if not finished(system):
        system.rotate(-1)
        system[0].inputs.append(comp.outputs[-1])
    return system

def thrust_with_feedback(phase):
    system = first_true(repeatedly(run_one_output, System(phase)), finished)
    return thrust(system)

thrust_with_feedback([9, 8, 7, 6, 5])

1219070632396864

In [280]:
phase = max(permutations(range(5, 10), 5), key=thrust_with_feedback)
thrust_with_feedback(phase)

7818398

## Day 8

In [307]:
pixels, = Input(8, line_parser=compose(digits, str.strip))
len(pixels)

15000

In [309]:
import numpy as np

def Image(pixels, width, height):
    return np.array(pixels).reshape((-1, height, width))

im = Image(pixels, 25, 6)
im.shape

(100, 6, 25)

In [313]:
def count(layer, val):
    return np.sum(layer == val)

layer = np.argmin(mapt(count, im, cycle([0])))
count(im[layer], 1) * count(im[layer], 2)

14

In [327]:
def stack(above, below):
    return np.where(above == 2, below, above)

def print_layer(layer):
    for row in layer:
        print(cat('O' if val == 1 else ' ' for val in row))

print_layer(reduce(stack, im))

O      OO OOOO  OO  O  O 
O       O O    O  O O  O 
O       O OOO  O    OOOO 
O       O O    O    O  O 
O    O  O O    O  O O  O 
OOOO  OO  OOOO  OO  O  O 


## Day 9

In [65]:
prog, = Input(9, line_parser=integers)
prog[:5]

(1102, 34463338, 34463338, 63, 1007)

In [66]:
ADD, MUL, INPUT, OUTPUT, JT, JF, LT, EQ, REL, STOP = (
    1, 2, 3, 4, 5, 6, 7, 8, 9, 99)

opcodes = {
    1: 'ADD', 2: 'MUL', 3: 'INPUT', 4: 'OUTPUT',
    5: 'JT', 6: 'JF', 7: 'LT', 8: 'EQ', 9: 'REL',
    99: 'STOP',
}

arities = {
    ADD: 3, MUL: 3, INPUT: 1, OUTPUT: 1,
    JT: 2, JF: 2, LT: 3, EQ: 3, REL: 1,
    STOP: 0
}

def Opcode(val):
    code = val % 100
    return (val % 100,
            [val // 100 % 10, val // 1000 % 10, val // 10000 % 10])
Opcode(1003)

(3, [0, 1, 0])

In [208]:
@dataclass
class Comp:
    prog: defaultdict
    inputs: deque
    outputs: deque
    pc: int = 0
    base: int = 0
    waiting: bool = False
    stopped: bool = False
        
    def __init__(self, prog, inputs, outputs=[]):
        self.prog = defaultdict(int, dict(enumerate(prog)))
        self.inputs = deque(inputs)
        self.outputs = deque(outputs)
    
    def step(self):
        prog, pc = self.prog, self.pc
        inputs, outputs = self.inputs, self.outputs
        
        code, modes = Opcode(prog[pc])
        arg1 = self.read(pc + 1, modes[0])
        arg2 = self.read(pc + 2, modes[1])
        if code == STOP: self.stopped = True
        if code == ADD: self.write(arg1 + arg2, pc + 3, modes[2])
        elif code == MUL: self.write(arg1 * arg2, pc + 3, modes[2])
        elif code == INPUT:
            if not inputs:
                self.waiting = True
                return
            self.write(inputs.popleft(), pc + 1, modes[0])
        elif code == OUTPUT: outputs.append(arg1)
        elif code == JT and arg1 != 0:
            self.pc = arg2
            return
        elif code == JF and arg1 == 0:
            self.pc = arg2
            return
        elif code == LT: self.write(int(arg1 < arg2), pc + 3, modes[2])
        elif code == EQ: self.write(int(arg1 == arg2), pc + 3, modes[2])
        elif code == REL: self.base += arg1
        self.pc += arities[code] + 1
        
    def read(self, loc, mode):
        '''
        Reads in loc from prog accounting for mode.
        '''
        arg = self.prog[loc]
        return (self.prog[arg]             if mode == 0 else
                arg                        if mode == 1 else
                self.prog[self.base + arg] if mode == 2 else
                None)
    
    def write(self, val, loc, mode):
        '''
        Writes argument to prog accounting for mode.
        '''
        arg = self.prog[loc]
        if mode == 0:
            self.prog[arg] = val
        elif mode == 2:
            self.prog[self.base + arg] = val
        else:
            raise ValueError('Bad mode in write')
            
    def input(self, val):
        self.waiting = False
        self.inputs.append(val)
    
    def run(self):
        while not self.waiting and not self.stopped:
            self.step()
        return self
    
    def output(self):
        return self.outputs[-1]
    
    def out(self):
        return list(self.outputs)

In [68]:
quine = [109,1,204,-1,1001,100,1,100,1008,100,16,101,1006,101,0,99]
c = Comp(quine, []).run()
assert list(c.outputs) == quine

tests = [
    ([109, -1, 4, 1, 99], -1),
    ([109, -1, 104, 1, 99], 1),
    ([109, -1, 204, 1, 99], 109),
    ([109, 1, 9, 2, 204, -6, 99], 204),
    ([109, 1, 109, 9, 204, -6, 99], 204),
    ([109, 1, 209, -1, 204, -106, 99], 204),
]
for test, out in tests:
    assert Comp(test, []).run().output() == out

In [70]:
comp = Comp(prog, [1]).run()
comp.output()

3765554916

In [71]:
comp = Comp(prog, [2]).run()
comp.output()

76642

## Day 10

In [105]:
space = Input(10, test=False)
oids = {(x, y)
        for y, row in enumerate(space)
        for x, val in enumerate(row)
        if val != '.'}
len(oids)

398

In [106]:
from math import gcd

@t.curry
def Ray(orig, to):
    dx = X(to) - X(orig)
    dy = Y(to) - Y(orig)
    return (dx // gcd(dx, dy), dy // gcd(dx, dy))

Ray((2, 3), (3, 4))

(1, 1)

In [107]:
colinears = { oid: t.groupby(Ray(oid), oids - {oid}) for oid in oids }

In [108]:
def detects(oid):
    return len(colinears[oid])

station = max(oids, key=detects)
station

(30, 34)

In [109]:
detects(station)

344

In [110]:
def angle_from_up(vec):
    '''Returns number in [-2, 2] monotonically increasing as the angle from
    (0, -1) increases. https://stackoverflow.com/a/16561333'''
    dx, dy = vec
    p = dy / (abs(dx) + abs(dy))
    return p - 1 if dx >= 0 else 1 - p

def station_dis(oid):
    return cityblock_distance(station, oid)

assert (sorted([LEFT, DOWN, UP, RIGHT], key=angle_from_up) ==
        [UP, RIGHT, DOWN, LEFT])

In [122]:
sees = colinears[station]
angles = sorted(sees.keys(), key=angle_from_up)
vaporizing = [sorted(sees[angle], key=station_dis) for angle in angles]
bet = nth(t.interleave(vaporizing), 200 - 1)
X(bet) * 100 + Y(bet)

2732

## Day 11

In [101]:
prog, = Input(11, line_parser=integers)
prog[:5]

(3, 8, 1005, 8, 335)

In [102]:
from dataclasses import field

BLACK, WHITE = 0, 1

@dataclass
class Painter:
    comp: Comp
    hull: defaultdict
    pos: tuple = origin
    heading: tuple = UP
    
    def __init__(self, prog, start_on=0):
        self.comp = Comp(prog, [start_on])
        self.hull = defaultdict(None, { origin: start_on })
    
    def move(self):
        self.comp.run()
        if not self.comp.stopped:
            self.paint()
            on = self.hull.get(self.pos, 0)
            self.comp.inputs.append(on)
        return self
    
    def paint(self):
        color, turn = self.comp.outputs[-2], self.comp.outputs[-1]
        self.hull[self.pos] = color
        self.heading = (turn_left(self.heading) if turn == 0 else
                        turn_right(self.heading))
        self.pos = add(self.pos, self.heading)
        
    def run(self):
        while not self.comp.stopped:
            self.move()
        return self

In [103]:
p = Painter(prog).run()
hull = p.hull

In [104]:
len(hull.values())

2415

In [105]:
p = Painter(prog, 1).run()
hull = p.hull

In [106]:
for row in range(6):
    print(cat('O' if hull.get((col, row), 0) == 1 else ' '
              for col in range(43)))

 OOO  OOOO OOO  O  O OOOO O  O OOO   OO    
 O  O O    O  O O  O    O O  O O  O O  O   
 OOO  OOO  O  O O  O   O  O  O O  O O      
 O  O O    OOO  O  O  O   O  O OOO  O      
 O  O O    O    O  O O    O  O O    O  O   
 OOO  O    O     OO  OOOO  OO  O     OO    


## Day 12

In [54]:
starts = Input(12, line_parser=integers)

def Moon(pos, vel=(0, 0, 0)):
    return pos, vel

pos = X
vel = Y

moons = mapt(Moon, starts)
moons

(((1, 4, 4), (0, 0, 0)),
 ((-4, -1, 19), (0, 0, 0)),
 ((-15, -14, 12), (0, 0, 0)),
 ((-17, 1, 10), (0, 0, 0)))

In [55]:
from pprint import pprint

def sign(p):
    return tuple(np.sign(p))

@t.curry
def gravity(m1, m2):
    'Returns m1 after applying velocity change from m2'
    pos1, vel1 = m1
    return Moon(pos1, add(vel1, sign(sub(pos(m2), pos1))))

def gravitate(moons):
    'Updates all pair-wise velocities in moons'
    def grav1(moons, other):
        return mapt(gravity(m2=other), moons)
    return reduce(grav1, moons, moons)

gravitate(moons)

(((1, 4, 4), (-3, -3, 3)),
 ((-4, -1, 19), (-1, 1, -3)),
 ((-15, -14, 12), (1, 3, -1)),
 ((-17, 1, 10), (3, -1, 1)))

In [56]:
def velocity(moon):
    'Returns moon after applying velocity'
    pos, vel = moon
    return Moon(add(pos, vel), vel)

def approach(moons):
    return mapt(velocity, moons)

simulate = compose(approach, gravitate)

def energy(moons):
    return np.abs(np.array(moons)).sum(axis=2).prod(axis=1).sum()

In [57]:
energy(repeat(1000, simulate, moons))

10635

For part 2, the first repetition must be a duplicate of the initial state since every state has a unique parent. We can find cycles for each axis independent of the others, then take the LCM of the cycle lengths for the three axis cycle lengths.

In [60]:
%%time

def on_axis(axis, moons):
    return map2d(lambda m: (m[axis],), moons)

@t.curry
def cycled(initial, state):
    _, moons = state
    return initial == moons

def simulation(axis):
    return tq(zip(count_from(1), repeatedly1(simulate, on_axis(axis, moons))))

def cycle_length(axis):
    index, _ = first_true(simulation(axis), cycled(on_axis(axis, moons)))
    return index 

cycle_lengths = mapt(cycle_length, [0, 1, 2])
print(cycle_lengths)

(186028, 231614, 108344)
CPU times: user 2min 36s, sys: 1.49 s, total: 2min 37s
Wall time: 2min 42s


In [61]:
np.lcm.reduce(cycle_lengths)

583523031727256

## Day 13

In [179]:
prog, = Input(13, line_parser=integers)
prog[:5]

(1, 380, 379, 385, 1008)

In [210]:
BLOCK = 2
PADDLE = 3
BALL = 4

def tiles(screen):
    'Returns tiles in reverse order since later values override earlier ones'
    return (screen[i-3:i] for i in range(len(screen), 0, -3))

In [211]:
arcade = Comp(prog, []).run()
quantify(tiles(arcade.out()), lambda t: t[2] == BLOCK)

427

Since the paddle moves at the same rate as the ball, we can just make the paddle's X coord always match the ball's X coord.

In [215]:
@t.curry
def locate(tile_id, screen):
    x, y, _ = first_true(tiles(screen), lambda tile: tile[2] == tile_id)
    return x, y

ball = locate(BALL)
paddle = locate(PADDLE)

def score(screen):
    x, y, score = screen[-3:]
    return score if x == -1 and y == 0 else False

def joystick(screen):
    ballx, bally = ball(screen)
    padx, pady = paddle(screen)
    return np.sign(ballx - padx)

In [216]:
arcade = Comp(prog, [])
arcade.prog[0] = 2

while not arcade.stopped:
    arcade.run()
    screen = arcade.out()
    arcade.input(joystick(screen))
score(screen)

21426

## Day 14

There are no duplicate output chemical recipes, so the reaction list describes a DAG. Processing chemicals in topological order guarantees that we won't need to process a chemical more than once, eliminating the need to keep track of "leftovers".

In [163]:
def Ing(atom):
    qty, chem = atom.split()
    return int(qty), chem

Rxn = namedtuple('Rxn', ['yld', 'inputs'])
def Requirement(line):
    inputs, out = line.strip().split(' => ')
    yld, chem = Ing(out)
    return { chem: Rxn(yld, mapt(Ing, inputs.split(', '))) }

reqs = t.merge(Input(14, line_parser=Requirement))
reqs['ORE'] = Rxn(1, tuple())
len(reqs)

57

In [164]:
def topo_sort(nodes, children_func):
    ordering = []
    seen = set()
    def visit(node):
        if node in seen: return
        mapt(visit, children_func(node))
        ordering.append(node)
        seen.add(node)
    mapt(visit, nodes)
    return tuple(reversed(ordering))

def inputs(chem):
    return mapt(t.get(1), reqs[chem].inputs)

ordering = topo_sort(reqs.keys(), inputs)

In [165]:
from math import ceil

def requires(chem, qty):
    yld, inputs = reqs[chem]
    multiple = ceil(qty / yld)
    return { input_chem: multiple * input_qty
             for input_qty, input_chem in inputs }

def produce(qties, chem):
    return t.merge_with(sum, qties, requires(chem, qties[chem]))

@t.curry
def ore(result, amount):
    return reduce(produce, ordering, {result: amount})['ORE']

ore('FUEL', 1)

612880

Instead of processing the graph backwards, I'll just run binary search on fuel amounts until I need 1 trillion ore. The fuel produced is lower-bounded by 1T / (ore for 1 FUEL).

In [167]:
def binary_search(lower, upper, goal, f=identity):
    'Finds largest integer x between lower and upper so that f(x) <= goal.'
    mid = (lower + upper) // 2
    return (False if lower > upper else
            lower if mid == lower else
            binary_search(mid, upper, goal, f) if f(mid) < goal else
            binary_search(lower, mid, goal, f) if f(mid) > goal else
            mid)

ORE = 1e12
one_fuel = ore('FUEL', 1)
required = ore('FUEL')

int(binary_search(ORE // one_fuel, 2 * ORE // one_fuel, ORE, required))

2509120

## Day 15

First, I use DFS to find all valid locations and goal location in the maze. Then, I use BFS to find the shortest path to the goal.

In [4]:
prog, = Input(15, line_parser=integers)
len(prog)

1045

In [53]:
start = (21, 21)
WALL, OK, GOAL = 0, 1, 2
inputs = { 1: UP, 2: DOWN, 3: LEFT, 4: RIGHT }
back = [None, 2, 1, 4, 3].index

def move(bot, inp):
    return bot.input(inp).run()

def move_back(bot, inp):
    return bot.input(back(inp)).run()

def explore(bot, start=start):
    seen = set()
    goal = set()
    
    def visit(loc):
        seen.add(loc)
        for inp, heading in inputs.items():
            to = add(loc, heading)
            if to in seen:
                continue
                
            move(bot, inp)
            if bot.output() == WALL:
                continue
                
            if bot.output() == GOAL:
                goal.add(to)
            visit(to)
            move_back(bot, inp)
    visit(start)
    return seen, first(goal)

seen, goal = explore(Comp(prog))

In [54]:
def print_maze():
    for row in range(40):
        print(cat('S' if start == (col, row) else
                  'G' if goal == (col, row) else
                  ' ' if (col, row) in seen else
                  'x' for col in range(40)))
# print_maze()

In [55]:
def maze_spaces(loc):
    return [add(loc, heading) for heading in HEADINGS
            if add(loc, heading) in seen]

len(bfs(start, maze_spaces, [goal])) - 1 # since we count actions, not states

244

For part 2, I just brute-force the maximum distance to the goal:

In [58]:
max(len(bfs(goal, maze_spaces, [loc])) - 1 for loc in seen)

278

## Day 16

In [166]:
n = digits(Input(16, line_parser=integers)[0][0])
len(n)

650

In [167]:
%%time

from math import ceil
base = np.array([0, 1, 0, -1])

n_digits = len(n)

@t.memoize
def pattern(pos):
    for_pos = np.repeat(base, pos)
    reps = ceil(n_digits / (len(for_pos) - 1))
    return np.tile(for_pos, reps)[1:n_digits + 1]

@t.curry
def phase(n, pat):
    return np.abs(np.sum(n * pat)) % 10

def fft(n):
    patterns = mapt(pattern, range(1, len(n) + 1))
    return mapt(phase(n), patterns)

ffted = repeat(100, fft, n)

CPU times: user 4.57 s, sys: 42 ms, total: 4.62 s
Wall time: 6.42 s


In [168]:
cat(mapt(str, ffted[:8]))

'94960436'

In part 2, we need to avoid unnecessary calculations. Since the offset occurs after the halfway point of the input, the only coefficients are 1s. This means we just calculate cumulative sums.

In [169]:
REPS = 10000

n_len = len(n) * REPS
offset = int(cat(mapt(str, n[:7])))
ns = np.array([n[index % len(n)] for index in range(offset, n_len)])
ns

array([6, 3, 0, ..., 3, 9, 7])

In [170]:
def simple_fft(ns):
    return np.cumsum(ns[::-1])[::-1] % 10

In [171]:
cat(mapt(str, head(repeat(100, simple_fft, ns), 8)))

'57762756'

## Day 17

In [172]:
prog, = Input(17, line_parser=integers)
len(prog)

1471

In [180]:
view = cat(mapt(chr, Comp(prog).run().out())).split()
scaffolds = {(col, row)
             for row, line in enumerate(view)
             for col, ch in enumerate(line)
             if ch != '.' }
intersects = {pos for pos in scaffolds
              if all(loc in scaffolds for loc in neighbors4(pos))}
sum(x * y for x, y in intersects)

7584

Printing the scaffolding shows that the robot can always go straight through every intersection until it reaches a dead-end. I'll find a sequence of moves for the robot, then break them up manually into three functions.

In [186]:
HEADINGS

((0, -1), (-1, 0), (0, 1), (1, 0))

In [189]:
loc = first((col, row)
            for row, line in enumerate(view)
            for col, ch in enumerate(line)
            if ch in '^v<>' )
facing = HEADINGS['^<v>'.index(view[Y(bot)][X(bot)])]
bot = loc, facing
bot

((32, 0), (0, -1))

In [200]:
def turn(bot):
    loc, facing = bot
    left = turn_left(facing)
    right = turn_right(facing)
    return (((loc, left), 'L') if add(loc, left) in scaffolds else
            ((loc, right), 'R') if add(loc, right) in scaffolds else
            ((loc, facing), False))

def walk(bot):
    loc, facing = bot
    for steps in count_from(0):
        if add(loc, facing) not in scaffolds:
            return (loc, facing), steps
        loc = add(loc, facing)

def dust(bot):
    commands = []
    while True:
        bot, direction = turn(bot)
        if not direction: return commands
        
        bot, steps = walk(bot)
        commands.append((direction, steps))

commands = dust(bot)
set(commands)

{('L', 8), ('L', 12), ('R', 4), ('R', 10), ('R', 12)}

In [257]:
encode = { c: '12345'[i] for i, c in enumerate(set(commands)) }
decode = { v: k for k, v in encode.items() }
encoded = cat(mapt(t.get(seq=encode), commands))
encoded

'152441541543243215244154154321524'

In [249]:
def replace(encoded, fns):
    if len(encoded) == 0: return ''
    for name, body in fns.items():
        if encoded.startswith(body):
            return f'{name}{replace(encoded[len(body):], fns)}'
    return encoded[0] + replace(encoded[1:], fns)

funcs = { 'A': '1524', 'B': '432', 'C': '415' }
replace(encoded, funcs)

'ACCBBACCBA'

In [291]:
line = ','.join

def decode_func(name):
    return [str(command) for ch in funcs[name] for command in decode[ch]]

def to_ascii(funcs):
    main = line(replace(encoded, funcs))
    func_a = line(decode_func('A'))
    func_b = line(decode_func('B'))
    func_c = line(decode_func('C'))
    program = '\n'.join([main, func_a, func_b, func_c]) + '\n'
    return mapt(ord, program + 'n\n')

program = to_ascii(funcs)

In [295]:
start = list(prog)
start[0] = 2
Comp(start, program).run().output()

1016738